In [1]:
from datetime import datetime
import os
from shutil import rmtree

import numpy as np
import pandas as pd
from pyproj import Transformer
from dataclasses import dataclass
from numpy.typing import NDArray
from typing import List


## Prepare data structures and coordinate extraction

In [2]:
@dataclass
class BoundingBox:
    min_x: float
    max_x: float
    min_y: float
    max_y: float
    min_z: float
    max_z: float

@dataclass
class ResultVariable:
    name: str
    units: str
    data: NDArray

@dataclass
class Resolution:
    x: float
    y: float
    z: float

@dataclass
class ResultCollection:
    name: str
    bounding_box: BoundingBox
    resolution: Resolution
    variables: List[ResultVariable]
    
    
def _convert_lat_lon_str_to_decimal(coordinate: str) -> float:
    """
    Convert a latitude or longitude string in Decimal Minutes format to a decimal.

    Explicitly this works with the formats found in the bottle file of: '66¡51.502N'

    Parameters
    ----------
    coordinate : str
        Input latitude or longitude

    Returns
    -------
    float
        Decimalised latitude or longitude
    """
    # If we were being strict, we'd do a regex check here first but it's just not necessary for the ad-hoc use case.
    major, minor = coordinate.split('¡')

    direction_multiplier = -1 if 'S' in minor or 'W' in minor else 1
    for char in ['N', 'S', 'E', 'W']:
        minor = minor.replace(char, '')

    return direction_multiplier * (float(major) + float(minor) / 60)


## Translate csv files into regular grids, extracting variables, extents and units

In [5]:
# Target CRS for the viewer - EPSG:3413
target_proj4 = "+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
# target_proj4 = "+proj=stere +lat_0=90 +lat_ts=75 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs"

# This is an artificial width for the sake of visualising a column of values in the front end.
width = 5000

# We have to interpolate onto a regular z axis. This sets how many samples to take vertically
nb_vertical_samples = 50

# Arbitrary date as the data is not supplied with one.
dates = [datetime(2022, 9, 9)]

# Build a list of result objects. 1 per file.

results = []

for file_id in ['1', '2']:
    
    # Open the file and figure out the basic extent details.
    bottle_file_path = os.path.join('example_data', f'Station_BB{file_id}.csv')

    # There are some funky bytes in these files requiring the non-standard encoding.
    df = pd.read_csv(bottle_file_path, encoding="ISO-8859-1")
    df.sort_values("Depth_m", inplace=True)

    depths = df.Depth_m.values

    # Simple interp values at a regular interval:
    depths_interpolated = np.linspace(depths.min(), depths.max(), nb_vertical_samples)
    vertical_resolution = (depths.max() - depths.min()) / (nb_vertical_samples - 1)

    # Take the coords from the file and covert to our desired crs
    transformer = Transformer.from_crs('epsg:4326', target_proj4)
    lat = _convert_lat_lon_str_to_decimal(df.Latitude[0])
    lon = _convert_lat_lon_str_to_decimal(df.Longitude[0])

    x, y = transformer.transform(lat, lon)

    bounding_box_3d = BoundingBox(min_x=x - width / 2, max_x=x + width / 2,
                                  min_y=y - width / 2, max_y=y + width / 2,
                                  min_z=-depths.max() - vertical_resolution / 2, max_z=-depths.min() + vertical_resolution / 2)

    result_variables = []
    result_collection = ResultCollection(
        name=f"bottle-station-{file_id}",
        bounding_box=bounding_box_3d,
        variables=result_variables,
        # As we synthesize extra x and y values below, we must adjust the x/y resolution accordingly.
        resolution=Resolution(x=width/2, y=width/2, z=vertical_resolution))
    results.append(result_collection)
    
    for field in ['Salinity_psu', 'Temperature_C']:

        raw_data = df[field].values

        # Note, the depths must be increasing for np.interp to work properly
        data = np.interp(depths_interpolated[::-1], depths, raw_data)

        # Convert data into [z, y, x] format
        data = np.expand_dims(np.expand_dims(data, axis=1), axis=2)
        # Work around an xcube limitation, for x, y axes to have more than 1 value:
        data = np.repeat(np.repeat(data, 2, axis=1), 2, axis=2)
        
        # csv convention has the column suffixed with units.
        units = field.split("_")[-1]
        data_set_name = f"manitoba-{field.replace('_' + units, '')}-{file_id}"
        
        result_variables.append(ResultVariable(name=data_set_name, units=units, data=data))


In [4]:
# Quickly check our result structure is as expected.
assert results[0].variables[0].units == 'psu'

## Now save in an xcube compatible format

In [6]:
import netCDF4
from rasterio.crs import CRS
import numpy as np
import xarray as xr
from xcube.core.chunk import chunk_dataset
from xcube.core.store import new_data_store

for result in results:
    with netCDF4.Dataset('tmp.nc', 'w', diskless=True) as nc:
        
        # Get first variable as a reference
        data = result.variables[0].data
        bounding_box = result.bounding_box
        resolution = result.resolution

        # Create dimensions
        nc.createDimension('z', data.shape[0])
        nc.createDimension('y', data.shape[1])
        nc.createDimension('x', data.shape[2])

        # Create variables
        z_var = nc.createVariable('z', np.float64, ('z'))
        z_var.standard_name = 'Height'
        z_var.axis = 'z'

        y_var = nc.createVariable('y', np.float64, ('y'))
        y_var.standard_name = 'North'
        y_var.axis = 'y'

        x_var = nc.createVariable('x', np.float64, ('x'))
        x_var.standard_name = 'East'
        x_var.axis = 'x'

        z = np.arange(bounding_box.min_z + resolution.z / 2.0, bounding_box.max_z, resolution.z)
        y = np.arange(bounding_box.min_y + resolution.y / 2.0, bounding_box.max_y, resolution.y)
        x = np.arange(bounding_box.min_x + resolution.x / 2.0, bounding_box.max_x, resolution.x)

        z_var[:] = z
        y_var[:] = y
        x_var[:] = x

        # Set up CRS
        crs = CRS.from_proj4(target_proj4)
        nc_crs = nc.createVariable('spatial_ref', 'i4')
        nc_crs.spatial_ref = crs.to_wkt()

        for variable in result.variables:
            var = nc.createVariable(variable.name, "float32", ('z', 'y', 'x'), fill_value=999)
            var.units = variable.units
            var.long_name = variable.name
            var.short_name = variable.name
            var[:, :, :] = variable.data
            var.grid_mapping = 'spatial_ref'

        chunk_size=256
        store = new_data_store('file', root='./xcube-server-data/')
        dataset = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))

        for variable in result.variables:
            dataset.variables[variable.name].attrs['4d_viewer_layer_type'] = 'heatmap3d'  # or heightmap

        dataset.attrs['4d_viewer_ui_path'] = '/example_data/'
        chunked_dataset = chunk_dataset(dataset, dict(z=chunk_size, y=chunk_size, x=chunk_size), format_name='zarr')

        store.write_data(chunked_dataset, f'{result.name}.zarr', replace=True)
        store.write_data(chunked_dataset, f'{result.name}.levels', replace=True,
                         base_dataset_id=f'{result.name}.zarr', agg_methods="first")        


/opt/conda/lib/python3.12/site-packages/xcube/util/plugin.py:176: UserWarning: Initializing xcube plugin 'xcube_4d_viewer' took 278 ms, consider code optimization. (For example, avoid eager import of packages, consider lazy loading of resources, etc.)
  warnings.warn(
